In [1]:
using constants
using types
using Optim
using PyPlot
using data
include("DOSfunctions.jl")
include("userfunctions.jl")
Eoff=2.0
Ts=[600.0]#collect(300.0:100.0:900.0)
xs=collect(0.0:0.01:1.0)
Es=collect(0.0:0.1:4.0)
x=0.6
var0=[Ts[1],x]
xmax=4.0
EX3(var)=Eoff#+Egx3(var[1],var[2])
EX1(var)=Ec0(var[2])+EX3(var)
ELH(var)=Eoff-Egx3(var[1],var[2])
EHH(var)=Eoff-Egx3(var[1],var[2])

effMassX1(var)=X1effMass(var[1])*me
effMassX3(var)=0.38*me
effMassLH(var)=-1.0*me
effMassHH(var)=-1.5*me

NII=1e12*1e6
bandX3=parBandTx(effMassX3(var0),0.0,EX3(var0),[EX3],[effMassX3],var0)
bandX1=parBandTx(effMassX1(var0),0.0,EX1(var0),[EX1],[effMassX1],var0)
bandHH=parBandTx(effMassHH(var0),0.0,EHH(var0),[EHH],[effMassHH],var0)
bandLH=parBandTx(effMassLH(var0),0.0,ELH(var0),[ELH],[effMassLH],var0)
bndstTx=BandStrucTx([bandLH,bandHH,bandX1,bandX1,bandX1,bandX3,bandX3,bandX3],var0)
vartau0=[bandX3.effMass,Ts[1],1.0,x,1.5,bandX3,NII,bndstTx]

tauAC(var)=tauAC_func(Cl(var[4]),acPhDa(var[4]),var[2],var[1],var[3],var[6])
tauPOP(var)=tauPOP_func(epsilon0_x(var[4]),epsilonhf_x(var[4]),var[6],var[5],var[2],var[1],var[3]
,tauPOPIIint(var[5],var[2],var[4],var[6]),var[7],opPhE(var[4]),var[8])
tauPOP2(var)=tauPOP2_func(epsilon0_x(var[4]),epsilonhf_x(var[4]),var[6],var[5],var[2],var[1],var[3]
,opPhE(var[4]))
tauII2(var)=tauII2_func(epsilon0_x(var[4]),epsilonhf_x(var[4]),var[6],var[5],var[2],var[7],var[1],var[3],var[8])
tauII(var)=tauII_func(epsilon0_x(var[4]),epsilonhf_x(var[4]),var[6],var[5],var[2],var[7],var[1],var[3]
,tauPOPIIint(var[5],var[2],var[4],var[6]))

tauTOTTx=tau_electron_B([tauAC,tauPOP2,tauII2],vartau0)#tauAC,tauII2,tauPOP2
numofn=[-3.0e19]#Sample 1 -3.0e19, Sample 2 -1.25e20, Sample 3 -1.75e20, Sample Sb 0.75% -1.03e20, #
bndstTx.var[1]=Ts[1]    
tauTOTTx.variables[2]=Ts[1]    
types.updatebnstTx(bndstTx)  

In [ ]:
function DOSrange(band::parBandTx,Es)
    n=length(Es)
    dos=Array{Float64}(n)
    for i in 1:n
        dos[i]=getDOS_SingleBand_E(band::parBandTx,Es[i])
    end
    return dos
end

In [ ]:
function integrandV(band::parBandTx,E,T,Ef)
    return (-1+fermiStat_Temp_Ef_E(T,Ef,E))*q*getDOS_SingleBand_E(band,E)#*(-1*fermiStat_Temp_Ef_E(T,Ef,E))
    
end

In [ ]:
function integrandVR(band::parBandTx,Es,T,Ef)
    n=length(Es)
    int=Array{Float64}(n)
    for i in 1:n
        int[i]=integrandV(bandLH,Es[i],600.0,Ef)
    end
    return int
end

In [ ]:
ns=integrandVR(bandLH,Es,600.0,2.2)

In [ ]:
plot(Es,ns)

In [ ]:
function integralV(integrand,band,T)
    min=band.offset-20kBe*T<0 ? 0.0 : band.offset-20kBe*T
    return quadgk(integrand,min,band.offset)
end

In [ ]:
integrandVf(x)=integrandV(bandLH,x,600.0,2.3)
integralV(integrandVf,bandLH,600.0)

In [ ]:
function integralRange(band,T,Efs)
    n=length(Efs)
    int=Array{Float64}(n)
    for i in 1:n
        integrandVf(x)=integrandV(band,x,T,Efs[i])
        @time int[i]=integralV(integrandVf,band,T)[1]
    end
    return int
    
end

In [ ]:
jj=integralRange(bandLH,600.0,Es)

In [ ]:
function integrandC(band::parBandTx,Es,T,Ef)
    return q*getDOS_SingleBand_E(band,E).*(1-fermiStat_Temp_Ef_E(T,Ef,E))
    
end

In [ ]:
function NsEfrange(bndstTx,Efsx,T)
    ns=Array{Float64}(length(Efsx))
    for (i,Ef) in enumerate(Efsx)
        ns[i]=NumofnMultiBand(bndstTx,Ef,600.0,3.0)
    end
    return ns
end

In [ ]:
Profile.init(delay=0.0001)

In [ ]:
NumofnMultiBand(bndstTx,2.0,300.0,3.0)

In [ ]:
Profile.clear()

In [ ]:
@profile NumofnMultiBand(bndstTx,2.0,300.0,3.0)

In [ ]:
Profile.print()

In [ ]:
using ProfileView

In [ ]:
ProfileView.view()

In [ ]:
function EfArray(Ts,bndstTx,tauTOTTx,numofn,xmax)
    Efs=Array{Float64}(length(Ts),length(numofn))
    for (j,Tx) in enumerate(Ts)        
        for (k,nx) in enumerate(numofn)
            tauTOTTx.variables[7]=nx*1e6
            @time Efs[j,k]=Fermilevel_n(nx,bndstTx,Tx,xmax) # Efinput[j]+Eoff#
            #numofnsout[j,k]=NumofnMultiBand(bndstTx,Efs[j,k],Tx,xmax)    
        end
    end
    return Efs
end

In [ ]:
EfArray(Ts,bndstTx,tauTOTTx,numofn,xmax)

In [ ]:
Profile.init(delay=0.01)

In [ ]:
EfArray(Ts,bndstTx,tauTOTTx,numofn,xmax)

In [ ]:
Profile.clear()

In [ ]:
@profile EfArray(Ts,bndstTx,tauTOTTx,numofn,xmax)

In [ ]:
Profile.print()

In [ ]:
using ProfileView

In [ ]:
ProfileView.view()

In [ ]:
Pkg.add("QuantEcon")

In [ ]:
function integranddeep(bndst,Ef,T,E)
    x1=q*getDOS_MultiBand_E_Total(bndst,E)
    x2=-fermiDerivativeTemp_Ef_E(Ef,T,E)
    println("E:$E,\n x1:$x1 ,\n x2:$x2\n")
    return x1.*x2
end
function integralDQK(bndst,Ef,T) 
    min=Ef-20kBe*T<0 ? 0.0 : Ef-20kBe*T
    integrand(x)= integranddeep(bndst,Ef,T,x)
    nodes, weights = qnwlege(65, min, (Ef+20kBe*T))
    return do_quad(integrand,nodes,weights)
end
Efqs=[2.01,2.0023,1.98,1.95,1.93,1.90,1.88]
integralDQK(bndstTx,Efqs[1],Ts[1])

In [ ]:
El=collect(1.5:0.01:2.5)
getDOS_MultiBand_E_Total(bndstTx,El)